# Introduction to xemc3

xemc3 is a library for reading the output fron EMC3 simulations into the xarray format.

In [ ]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import xemc3

# Matplotlib setup
import setup_plt

## Quick introduction to xarray

This is just to get a quick introduction of the structure of the xarray data type.

In the cell below we generate a xarray with dimensions $(3,3)$ for variable $x$ with coordinates $(10,20)$ and $y$ with coordinates $(1,2,3)$.

In [ ]:
data = xr.DataArray(
    np.random.rand(2, 3), dims=("x", "y"), coords={"x": [10, 20], "y": [1, 2, 3]}
)
data

### data.values

Returns the wrapped numpy array, in this case the return value of `np.random.rand(2, 3)`

In [ ]:
data.values

### data.dims

Returns the name of the dimensions.

In [ ]:
data.dims

### data.coords
Returns the coordinates for all axis directions with coordinate names and datatype of the coordinates.

In [ ]:
data.coords

### data.attrs

Returns other attributes in form of a dictionary with you can easily add by generating a new value associated with a new key.

In [ ]:
data.attrs["units"] = "m"
data.attrs

### More on xarray:
* [Documentation](https://xarray.pydata.org/en/stable/index.html)
* [Tutorials](https://xarray-contrib.github.io/xarray-tutorial/index.html)

## EMC3 data

The prerequisite for this example to work is to have downloaded the file emc3_example.nc and have the libraries specified in this script installed in your enviroment. We recommend using netCDF4 for opening .nc files. The emc3_example.nc can be found and downloaded here: https://gitlab.mpcdf.mpg.de/dave/xemc3-data given that you have acces.

The path specified in the string in the cell below is where you have stored the emc3_example.nc locally on your computer.

In [ ]:
# Use local helper function to get some data
from get_data import load_example_data

ds = load_example_data()
# If you want to use your own data use something like
# ds = xemc3.load.all("path/to/mydata/")
# or if you have converted it already to a netcdf file
# ds = xr.open_dataset("path/to/mydata.nc")

ds

### dataset (ds) explanation

When running the codeline ds on the last line of a cell you get an overview of what the xarray object consist of.

#### ds.coords['R_bounds']

`R_bounds` represents the coordinates of the vertices at the gridcells in the radial direction in the $xy$-plane. with $R = \sqrt{x^2 + y^2}$.

#### ds.coords['z_bounds']
`z_bounds` represents the coordinates of the vertices of the gridcells in the $z$-direction.

#### ds.coords['phi_bounds']
`phi_bounds` represents the coordinates of the vertices of the gridcells in the $\phi$-direction.

In [ ]:
# The shape is 6 dimensional, as we include 3 dimensions for the vertices
ds.coords["R_bounds"].shape

In [ ]:
# Note that units are m - xemc3 prefers SI, with only eV as exception
ds.coords["z_bounds"]

### Toroidal slice
A toroidal slice is defined as the grid of $(R,z)$-values at a fixed angle $\phi$. The values of the $\phi$-angles used in the W7X grid can be found in the paragraph below and demonstrated in the next cell.

### ds.coords['phi_bounds']
Running the cell below gives you an array of the $\phi$ angles.

In [ ]:
ds.coords["phi_bounds"]

### ds.emc3.plot_rz(key, phi = $\phi$)

The key is given as a string, `None` can be passed as a key if you want to plot the mesh. An example is the angle `phi` $= \phi$ which is the angle given in radians as float. 

For this particular example(.nc file) the floats of the angle $\phi$ can be found in the dictionary defined by ds.coords['phi_bounds'] which has 2 dimensions; one for either side of the cell for a given angle $\phi$. There are 37 different values for $\phi$ since W7-X has a five-fold symmetry which is divided in two up-down symmetric parts and we use a resolution $1^{\circ}$.

In the cells below are some examples of the parameter electron temperature $T_e$ plotted in toroidal slices for phi index $n_{\phi} = [0,18,35]$.

In [ ]:
# the parameter can be plotted by a one-liner
plt.figure()
ds.emc3.plot_rz("Te", phi=0)

In [ ]:
# for several angles and control
import ipywidgets as widgets

fig = plt.figure()


def plot_Te(ip):
    fig.clear()
    ds.emc3.plot_rz("Te", phi=ip)


ip = widgets.FloatSlider(min=0, max=np.pi / 5, value=0, step=0.01)
widgets.interact(plot_Te, ip=ip)

### Simplifying of grid structure

The parameter values are defined in each grid cell, but the center or the mean of the vertices of the gridcell: $\mathbf{r}_{param} = \langle \mathbf{r}_{vertex} \rangle$. A simplified analogy is the centerpoint of a 3D cube.

You can get the center coordinates `ds.<direction>_bounds.mean(dim=("delta_r", "delta_theta", "delta_phi"), ignore_missing=True)` by averaging over the `delta_*` dimensions of the variable, as that contains the cell extend in each direction.

In [ ]:
R = ds.R_bounds.mean(dim=("delta_r", "delta_theta", "delta_phi"))
z = ds.z_bounds.mean(dim=("delta_r", "delta_theta", "delta_phi"))
phi = ds.phi_bounds.mean(dim="delta_phi")
x = R * np.cos(phi)
y = R * np.sin(phi)
x, ds.Te

### Use of NaN values in xEMC3

Not all gridcells have a defined parameter value attached to it. This is mostly the outer and inner region of the grid where the values of many parameter has been ignored because in the specific regions the quantity is not evolved.
This is illustrated in the above plot example of the electron temperature $T_e$. In the cell below you can see how large a fraction of the total number of gridpoints the mesh for the electron temperature that has NaN as a value.

In [ ]:
n_nans_Te = np.sum(np.isnan(ds.Te)).data
print("How many nans in Te field?", n_nans_Te)
print(f"Fraction of nans with respect to gridcells: {n_nans_Te/ds.Te.size*100:.2f} %")

### Grid

In the cell below there is an interactive plot of the grid. You can use the slider to iterate through all toroidal slices(all $\phi$ angles).

In [ ]:
import ipywidgets

fig = plt.figure()


def plot_emc3grid(ip):
    fig.clear()
    ds.emc3.plot_rz(None, phi=ip)


ip = ipywidgets.FloatSlider(min=0, max=np.pi / 5, value=0, step=0.01)
ipywidgets.interact(plot_emc3grid, ip=ip)

### Grid with boundaries

Interactive plot of the grid, here you can use the ipywidget slider to iterate through all toroidal slices,
the rmin and rmax to set the boundaries in r direction, and the zmin and zmax to set the boundaries in z direction.

In [ ]:
plt.figure()
ds.emc3.plot_rz("Te", phi=np.pi / 5, Rmin=5.8, Rmax=6.1, zmin=0, zmax=0.3)

### Periodic boundary conditions for plotting

Naturally the data does not have periodic boundary conditions, which means that the last dataframe would be equal to the first. In the case of emc3 data the periodicity is in the theta direction. For plotting the dimension of the theta grid is increased by one and set to the first values in the theta direction. This is for tha plot to "complete the orbit" in the theta direction for it to be closed.

In [ ]:
fig = plt.figure()


def plot_Te_zoomed(ip):
    fig.clear()
    c = plt.pcolormesh(
        ds.emc3["R_corners"][:, :, ip],
        ds.emc3["z_corners"][:, :, ip],
        ds.Te[:, :, ip],
        cmap=plt.cm.jet,
        shading="auto",
    )
    plt.colorbar(c)


phislider = ipywidgets.IntSlider(min=0, max=35)
ipywidgets.interact(plot_Te_zoomed, ip=phislider)